In [101]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.base import TransformerMixin
from sklearn.feature_selection import SelectKBest
import itertools
from sklearn.linear_model import SGDRegressor
from sklearn.cross_validation import KFold


import pandas
import numpy as np
from sklearn.grid_search import GridSearchCV
dfNotNull = pandas.read_csv('Long.csv')
#dfNotNull = dfNotNull.drop(dfNotNull.index[[218]])
dfNotNull.head()
print dfNotNull.Procedure3.value_counts()

Lumbar Radiofrequency                    224
Lumbar ESI                               145
Lumbar Medial Branch Block               113
Facet Joint Injection                     70
Cervical/Thoracic Medial Branch Block     43
Spinal Cord Stimulator                    27
Cervical/Thoracic ESI                     25
Cervical/Thoracic Radiofrequency          19
Joint Injection, other                     9
Nerve Block                                8
Not Assigned                               1
Name: Procedure3, dtype: int64


In [117]:
print "get null scores"

import CleanDefs

"""
print "Null Scores Based on Total Mean"
y = dfNotNull['TotalTimeMin']
x = np.repeat([dfNotNull['TotalTimeMin'].mean()], len(dfNotNull))
differences = (y - x).abs()
print differences.mean()
print np.sqrt((differences**2).mean())



print "Null Scores Based on Procedure Mean"


y = dfNotNull['TotalTimeMin']
X = dfNotNull['Procedure3']
pipe= Pipeline([('AvgProcTrsnfrm', AverageProcedureTransformer())])
print np.sqrt(-cross_val_score(pipe, X, y, cv=5, scoring='mean_squared_error').mean())

 """



numFolds = 5

y = dfNotNull['TotalTimeMin']
X = dfNotNull['Procedure3']

abse =[]
sqrd =[]
for i in range(1,25):
    kf = KFold(len(X), numFolds, shuffle=True)
    for train_indices, test_indices in kf:
        features_train = X[train_indices]
        response_train= y[train_indices]
        features_test = X[test_indices]
        response_test = y[test_indices]

        dfT = pandas.DataFrame()
        dfT['Procedure3'] = features_train
        dfT['TotalTimeMin']= response_train
        dfT['AverageProcTime']=dfT['Procedure3'].apply(lambda x: CleanDefs.createProcAvgs(x,dfT) )
        dfR = pandas.DataFrame()
        dfR['Procedure3'] = features_test
        dfR['TotalTimeMin']= response_test
        dfR['AverageProcTime']=dfR['Procedure3'].apply(lambda x: CleanDefs.createProcAvgs(x,dfT) )

        y1 = dfR['TotalTimeMin']
        X1 = dfR['AverageProcTime']
        differences = (y1 - X1).abs()
        abse += [differences.mean()]
        sqrd += [(differences**2).mean()]
print np.array(abse).mean()
print np.sqrt(np.array(sqrd)).mean()




get null scores
6.56750199776
10.2422627522


In [62]:
print "Different Features sets and corr plots"

Procedure3feat = ['Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Facet Joint Injection', 'Cervical/Thoracic ESI'] 
Procedure3feat += ['Cervical/Thoracic Medial Branch Block', 'Spinal Cord Stimulator', 'Nerve Block', 'Lumbar Medial Branch Block']
Procedure3feat += ['Lumbar ESI', 'Joint Injection, other']
                  
Procedure4feat = ['Radiofrequency','ESI', 'Medial Branch Block', 'Injection',  'Nerve Block 2','Spinal Cord Stimulator 2']
                  
ClassFeat = ['3', 'Gender', '1r']

NonScaledContfeat  = ['Age','BMI','HealthCodesLen', 'SurgicalHistoryLen']

ScaledContfeat  = ['AgeScaled','BMIScaled','HealthCodesLenScaled', 'SurgicalHistoryLenScaled']

HealthCodes = ['M54', 'G25', 'M51', 'M50', 'M53', 'G54', 'E78', 'C177', 'D51', 'G3', 'G2', 'N28', 'I73', 'M48', 'M43', 'M41', 'M46', 'M47', 'Z79', 'F32', 'G57', 'D68', 'Q76', 'Z72', 'Y92', 'T85', 'I69', 'C182', 'M32', 'G40','F03', 'G47', 'C178', 'Y83', 'Y84', 'I50', 'C45', 'E05', 'M25', 'F12', 'F17', 'K74', 'I48', 'I42', 'C79', 'I25','E11', 'J45', 'C6', 'L40', 'E21', 'C189', 'N1', 'M96', 'M10', 'M12', 'M19', 'F41', 'Z01', 'J44', 'M79', 'J43', 'Z80','I89', 'Z85', 'Z87', 'Z86', 'Z89', 'Z88', 'C95', 'I1', 'N40', 'M84', 'M81', 'M06', 'G95', 'G90', 'R53', 'R56', 'E06', 'E07', 'K21', 'E03', 'Z96', 'Z94', 'Z95', 'Z92', 'Z90', 'Z91', 'Z98', 'E89', 'C81', 'G82', 'R26', 'Q61']

InsuranceCodes = ['BLUE CROSS','BLUE SHIELD','MEDICARE']
print dfNotNull.TotalTimeMin.mean()
sigCodesList = []
for i in HealthCodes:
    if len(dfNotNull[dfNotNull[i] == 1]) > 20 and abs(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean() - 34.18) >=3 :
        print i + " = " + str(dfNotNull[dfNotNull[i] == 1].TotalTimeMin.mean()) + ", " + str(len(dfNotNull[dfNotNull[i] == 1]))
        sigCodesList += [i]
    
feature_cols = Procedure3feat + Procedure4feat + ClassFeat + NonScaledContfeat + sigCodesList
corrDF = dfNotNull[ NonScaledContfeat + ['TotalTimeMin']]
corrDF.corr()
print sigCodesList

Different Features sets and corr plots
34.1856725146
F32 = 42.347826087, 23
F17 = 31.0612244898, 98
M96 = 43.0961538462, 104
Z85 = 40.0833333333, 36
Z87 = 37.2727272727, 22
Z95 = 37.5675675676, 37
['F32', 'F17', 'M96', 'Z85', 'Z87', 'Z95']


In [3]:
print"""
Run KNN Classification ****
"""

#shortDF = dfNotNull[(dfNotNull['MinClass'] == "29 Max Class") | (dfNotNull['MinClass'] == "39 Max Class") | (dfNotNull['MinClass'] == "49 Max Class") | (dfNotNull['MinClass'] == "19 Max Class") |(dfNotNull['MinClass'] == "59 Max Class" )|(dfNotNull['MinClass'] == "69 Max Class")]
#feature_cols = [3,'Lumbar Radiofrequency','Nerve Block','Cervical/Thoracic Radiofrequency', 'Facet Joint Injection','Cervical/Thoracic ESI', 'Cervical/Thoracic Medial Branch Block','Spinal Cord Stimulator']
from sklearn.grid_search import GridSearchCV
feature_cols = Procedure3feat + Procedure4feat + ClassFeat + ScaledContfeat + sigCodesList
X = dfNotNull[feature_cols]
y = dfNotNull['MinClass']

""""for i in range (1,25):
    knn = KNeighborsClassifier(n_neighbors=i)
    scores = cross_val_score(knn, X, y, cv=3, scoring='accuracy')
    print i
    print scores.mean()"""
k_range = range(1, 30, 2)
param_grid = dict(n_neighbors=k_range)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=4).fit(X,
y)

grid.grid_scores_
grid_mean_scores = [result[1] for result in grid.grid_scores_]
print grid_mean_scores



Run KNN Classification ****

[0.48976608187134502, 0.50438596491228072, 0.51608187134502925, 0.53801169590643272, 0.54239766081871343, 0.55116959064327486, 0.56578947368421051, 0.56871345029239762, 0.58187134502923976, 0.58333333333333337, 0.59064327485380119, 0.57602339181286555, 0.57309941520467833, 0.56578947368421051, 0.56871345029239762]


C:\Users\James\Anaconda2\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=4.
  % (min_labels, self.n_folds)), Warning)


In [219]:
print"""
Run Decison-Tree Classification ****
"""    



X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']


depth_range = range(1, 30)
param_grid = dict(max_depth=depth_range)
treereg = DecisionTreeRegressor(max_depth=i,random_state=1)
grid = GridSearchCV(treereg, param_grid, cv=4).fit(X,
y)
grid.grid_scores_
grid_mean_scores = [result[1] for result in grid.grid_scores_]
print grid_mean_scores

print grid.best_score_




Run Decison-Tree Classification ****

[0.4541656081895129, 0.57739468851304598, 0.49859907493056094, 0.44924615299961768, 0.49525047919543735, 0.54312646340690596, 0.4317984105283289, 0.43265796871084006, 0.41830112923701362, 0.39739296498100313, 0.38217480370115681, 0.40935078588007706, 0.34374682409965618, 0.3766042960796252, 0.41117669266474632, 0.3302121583252941, 0.24692413955461665, 0.33133920841461217, 0.43191578306861644, 0.28108058913608819, 0.34703565534864411, 0.40269038475334301, 0.40269038475334301, 0.40269038475334301, 0.40269038475334301, 0.40269038475334301, 0.40269038475334301, 0.40269038475334301, 0.40269038475334301]
0.577394688513


In [10]:
print """
Run Linear Regression Train-Test Split ****
"""
feature_cols += ['Spinal Cord Stimulator']
X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']


abse = []
sqrd = []
for i in range(1,2):
    features_train, features_test, response_train, response_test \
    = train_test_split(X, y, random_state=4)
    # instantiate and fit

    
    linreg = LinearRegression()
    linreg.fit(features_train, response_train)
    print type(features_test)
    predictionDF = pandas.DataFrame()
    predictionDF['Predictions'] = linreg.predict(features_test)
    predictionDF['Actual'] = response_test.values
    differences = (predictionDF['Predictions'] - predictionDF['Actual']).abs()
    abse += [differences.mean()]
    sqrd += [(differences**2).mean()]
print np.array(abse).mean()
print np.sqrt(np.array(sqrd)).mean() 
f = lambda x:int(round(x/15.)*15)


print predictionDF.head(15)
print zip(feature_cols,linreg.coef_) 



Run Linear Regression Train-Test Split ****

<class 'pandas.core.frame.DataFrame'>
6.09559734025
9.29382480413
    Predictions  Actual
0     19.872998      18
1     29.632889      26
2     23.859569      23
3     19.872998      27
4     25.646317      27
5     31.039443      44
6     37.028868      31
7    101.943957     102
8     42.978185      49
9     25.646317      20
10    42.978185      36
11    40.770401      42
12    31.039443      25
13    42.978185      51
14    28.831659      37
[('Lumbar Radiofrequency', 5.9493174695004312), ('3', 1.7787869822836804), ('Radiofrequency', -51.971131993865313), ('ESI', -65.140430880427076), ('Medial Branch Block', -57.960556635538822), ('Injection', -59.367111053360595), ('Nerve Block 2', -61.977989848280245), ('1r', -2.2077846265808923), ('Spinal Cord Stimulator', 12.943957223988743)]


In [10]:
print """
Run Linear Regression w/ Select K Best feature Selection ****
"""
feature_cols = Procedure3feat  + ClassFeat + NonScaledContfeat + InsuranceCodes +Procedure4feat +sigCodesList
Procedure3feat + Procedure4feat + ClassFeat + NonScaledContfeat

X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']

linreg2 = LinearRegression()

def cross_val_Lin_KBest(X, y):
    filter1 = SelectKBest(k=i) # select the best i # of features
    
    linreg2 = LinearRegression()
    pipe = Pipeline([('anova', filter1), ('linreg', linreg2)])
    scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')
    
    mean2escores = cross_val_score(pipe, X, y, cv=5, scoring='mean_squared_error')
    meanAbsScores= cross_val_score(pipe, X, y, cv=5, scoring='mean_absolute_error')
    pipe.fit(X,y)
    tf_features =str(pipe.named_steps['anova'].get_support())
    print feature_list_creator(tf_features)
    return (scores,mean2escores,meanAbsScores) # return average R2

for i in range (1, len(feature_cols)+1):
    cvScores= cross_val_Lin_KBest(X, y)
    r2Scores = cvScores[0]
    mean2escores = cvScores[1]
    meanAbsScores= cvScores[2]
    
    print "root mean squared error"
    print np.mean(np.sqrt(-mean2escores))
    print "mean Abs Scores"
   
    print -meanAbsScores.mean()
    print '\n'








Run Linear Regression w/ Select K Best feature Selection ****

['Spinal Cord Stimulator 2']
root mean squared error
12.9101535678
mean Abs Scores
9.62836620825


['Spinal Cord Stimulator', 'Spinal Cord Stimulator 2']
root mean squared error
12.9081942242
mean Abs Scores
9.63956833543


['Spinal Cord Stimulator', 'Radiofrequency', 'Spinal Cord Stimulator 2']
root mean squared error
10.6824056043
mean Abs Scores
6.99285941705


['Lumbar Radiofrequency', 'Spinal Cord Stimulator', 'Radiofrequency', 'Spinal Cord Stimulator 2']
root mean squared error
10.6588595954
mean Abs Scores
6.90366596952


['Lumbar Radiofrequency', 'Spinal Cord Stimulator', 'Radiofrequency', 'ESI', 'Spinal Cord Stimulator 2']
root mean squared error
10.3569458932
mean Abs Scores
6.58426998297


['Lumbar Radiofrequency', 'Spinal Cord Stimulator', 'Lumbar ESI', 'Radiofrequency', 'ESI', 'Spinal Cord Stimulator 2']
root mean squared error
10.3549488744
mean Abs Scores
6.58723506741


['Lumbar Radiofrequency', 'Spinal Cor

In [12]:

        
        
        
print """
Run Linear Regression w/ Iteartive Feature Selection ****
"""
feature_cols = Procedure3feat  + ClassFeat + NonScaledContfeat  +Procedure4feat +sigCodesList

X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']



def cross_val_Lin_ITER(X, y):
    
    
    linreg2 = LinearRegression()
   
    
    mean2escores3 = cross_val_score(linreg2 , X, y, cv=5, scoring='mean_squared_error')
    meanAbsScores3= cross_val_score(linreg2 , X, y, cv=5, scoring='mean_absolute_error')
    return (mean2escores3,meanAbsScores3) # return average R2

mean2escorelist = []
for L in range(5, 11):#len(feature_cols)-8):
    print L
    print mean2escoresB 
    
   
    for subset in itertools.combinations(feature_cols, L):
        
        Subset = list(subset)
        X = dfNotNull[Subset]
        cvScores= cross_val_Lin_ITER(X, y)
        
        mean2escores = cvScores[0]
        mean2escores = np.mean(np.sqrt(-mean2escores))
        meanAbsScores= cvScores[1]
        meanAbsScores= -meanAbsScores.mean()
        
        if mean2escores < 10.35:
            mean2escorelist+= [mean2escores]
            
            print subset    
            print "root mean squared error"
            
            print mean2escores
            print "mean Abs Scores

            print meanAbsScores
            print '\n'
            



Run Linear Regression w/ Iteartive Feature Selection ****

5
18
('Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Spinal Cord Stimulator', 'Lumbar ESI', '3')
root mean squared error
10.3430553631
mean Abs Scores
6.60736419976


('Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Spinal Cord Stimulator', 'Lumbar ESI', '1r')
root mean squared error
10.3270157534
mean Abs Scores
6.6421238012


('Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Spinal Cord Stimulator', '3', 'ESI')
root mean squared error
10.2595945782
mean Abs Scores
6.50420755076


('Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Spinal Cord Stimulator', '1r', 'ESI')
root mean squared error
10.2575208136
mean Abs Scores
6.53783318404


('Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Spinal Cord Stimulator', 'Age', 'ESI')
root mean squared error
10.3342088563
mean Abs Scores
6.55618201388


('Lumbar Radiofrequency', 'Cervical/Thoracic Radiofrequency', 'Spina

KeyboardInterrupt: 

In [99]:
print "Standar Linear Regression W/ Cross Validation - Only using Gridsearch because CV was spazzing out"

#features which gave best found score in iterative feature assesment

feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection', 'Medial Branch Block',  '3','1r']#
feature_cols += ['Gender'] +['F17'] + ['Age'] 
tf= dfNotNull[feature_cols]
X = tf
y = dfNotNull['TotalTimeMin']



    
    
linreg3 = LinearRegression()

param_grid = dict()
mean2escores3 = GridSearchCV(linreg3, param_grid, cv=5, scoring='mean_squared_error').fit(X,y)
meanAbsScores3= GridSearchCV(linreg3, param_grid, cv=5, scoring='mean_absolute_error').fit(X,y)

mean2escores = mean2escores3.best_score_
print mean2escores
mean2escores = np.mean(np.sqrt(-mean2escores))
meanAbsScores= meanAbsScores3.best_score_
meanAbsScores= -meanAbsScores.mean()


print feature_cols   
print "root mean squared error"
            
print mean2escores
print "mean Abs Scores"

print meanAbsScores
print '\n'





Standar Linear Regression W/ Cross Validation - Only using Gridsearch because CV was spazzing out
-102.364970208
['Lumbar Radiofrequency', 'Radiofrequency', 'ESI', 'Nerve Block 2', 'Injection', 'Medial Branch Block', '3', '1r', 'Gender', 'F17', 'Age', 'MEDICARE']
root mean squared error
10.1175575219
mean Abs Scores
6.40162202414




In [90]:
print "Standard SDGC W/ GridSearch"
feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection', 'Medial Branch Block',  '3','1r']#
feature_cols += ['Gender'] #+['F17'] + ['Age'] 
tf= dfNotNull[feature_cols]
X = tf
y = dfNotNull['TotalTimeMin']

sgd = SGDRegressor()
print sgd.get_params().keys()


param_grid = {'loss':['squared_loss'], 
              'penalty':['none', 'l2', 'l1', 'elasticnet'],
              'alpha': [1e-4,1e-3,1e-2,1e-1,1,1e1],
              'n_iter': [10,100,1000]
             }

grid2= GridSearchCV(sgd , param_grid, cv=5, scoring='mean_absolute_error').fit(X,y)
grid1 = GridSearchCV(sgd, param_grid, cv=5, scoring='mean_squared_error').fit(X,y)
print 555
print  mean2escores3

mean2escores = np.sqrt(-grid1.best_score_)
meanAbsScores3 = -grid2.best_score_
print "root mean squared error"
print mean2escores

print "mean Abs Scores"
print meanAbsScores


Standard SDGC W/ GridSearch
['warm_start', 'loss', 'eta0', 'verbose', 'shuffle', 'fit_intercept', 'l1_ratio', 'average', 'n_iter', 'penalty', 'power_t', 'random_state', 'epsilon', 'alpha', 'learning_rate']
555
GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['none', 'l2', 'l1', 'elasticnet'], 'loss': ['squared_loss'], 'n_iter': [10, 100, 1000], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='mean_squared_error',
       verbose=0)
root mean squared error
10.4420860245
mean Abs Scores
6.40786359836


In [9]:
print mean2escores

nan


In [50]:
print """
Run KNN Regression Cross Val All Feature Combos Piped and non piped Scalar ****
"""


"scaling pipe"
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union

from sklearn.neighbors import KNeighborsRegressor
"Scale values in the DF"   
from sklearn.base import TransformerMixin
#from sklearn import preprocessing

class FeatureScalarTransformer(TransformerMixin):
        def transform(self,X,  **transform_params):
            if 'BMI' in X:
                #X['BMI'] = preprocessing.scale(X['BMI'])
                scal = StandardScaler().fit(X['BMI'])
                X['BMI'] = scal.transform(X['BMI'])
            if 'Age' in X:
                #X['Age']= preprocessing.scale(X['Age'].astype('float'))
                scal = StandardScaler().fit(X['Age'])
                X['Age']=  scal.transform(X['Age'])
            if 'SurgicalHistoryLen' in X:
                #X['SurgicalHistoryLen'] = preprocessing.scale(X['SurgicalHistoryLen'].astype('float')
                scal = StandardScaler().fit(X['SurgicalHistoryLen'])                
                X['SurgicalHistoryLen'] = scal.transform(X['SurgicalHistoryLen'])
            if 'HealthCodesLen' in X:
                #X['HealthCodesLen'] = preprocessing.scale(X['HealthCodesLen'].astype('float'))
                scal = StandardScaler().fit(X['HealthCodesLen'])
                X['HealthCodesLen'] = scal.transform(X['HealthCodesLen'])
            
            
            return X
           

        def fit(self, X, y=None, **fit_params):
            return self



 


Run KNN Regression Cross Val All Feature Combos Piped Scalar ****



In [25]:
def feature_list_creator(tf_features):
    b = str(tf_features).replace('[', 'r').replace("  "," ").replace("\n","").split(" ")
    b =[x for x in b if x != 'r']
    feature_list = []
    for num,i in enumerate(b):
        if 'Tr' in i:
            feature_list += [feature_cols[num]]
        else:
            pass
    return feature_list

In [44]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
print "Run KNN regression without scalar and with cross validation xyy"


feature_cols = Procedure3feat  + ClassFeat + ScaledContfeat + InsuranceCodes +Procedure4feat +sigCodesList#HealthCodes# 

X = dfNotNull[feature_cols]
y = dfNotNull['TotalTimeMin']
for i in range (1, len(feature_cols)+1):
    #gridsearch with select kbest
    neighbs_k = range(1, 30)
    param_grid = dict(neigh__n_neighbors=neighbs_k )
    neigh = KNeighborsRegressor()
    filter1 = SelectKBest(k=i) # select the best 2 features
    pipe = Pipeline([('anova', filter1), ('neigh', neigh)])
    grid1 = GridSearchCV(pipe, param_grid, cv=5, scoring='mean_squared_error').fit(X,y)
    grid2= GridSearchCV(pipe, param_grid, cv=5, scoring='mean_absolute_error').fit(X,y)
    tf_features=str(grid1.best_estimator_.named_steps['anova'].get_support())
    abs_features=str(grid2.best_estimator_.named_steps['anova'].get_support())
    print "Sqrt Mean squared error"
    print feature_list_creator(tf_features)
    
    print np.sqrt(-grid1.best_score_)
    print "Mean Absoulte error"
    print feature_list_creator(abs_features)
    print -grid2.best_score_
    print grid1.best_estimator_
    print " "
    
    
   
    
   

Run KNN regression without scalar and with cross validation xyy
Sqrt Mean squared error


NameError: name 'feature_list_creator' is not defined

In [36]:
print "GET BEST KNN PREDICTIONS"

feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection','Medial Branch Block' ]#
feature_cols += ['Spinal Cord Stimulator']
X = dfNotNull[feature_cols]

neighbs_k = range(1, 30)
param_grid = dict(neigh__n_neighbors=neighbs_k )
neigh = KNeighborsRegressor()

pipe = Pipeline([ ('neigh', neigh)])
grid1 = GridSearchCV(pipe, param_grid, cv=5, scoring='mean_squared_error').fit(X,y)
print np.sqrt(-grid1.best_score_)
KNNPredictions = grid1.predict(X)
print KNNPredictions[:5]

GET BEST KNN PREDICTIONS
10.3692137347
[ 32.5     29.625   40.75    25.1875  90.9375]


['3', 'Gender', 'Lumbar Radiofrequency', 'Nerve Block', 'Cervical/Thoracic Radiofrequency', 'Facet Joint Injection', 'Cervical/Thoracic ESI', 'Cervical/Thoracic Medial Branch Block', 'Spinal Cord Stimulator', 'SurgicalHistoryLen', 'BMI', 'Age', 'HealthCodesLen', 'Radiofrequency', 'ESI', 'Medial Branch Block', 'Injection', 'Spinal Cord Stimulator 2']


In [133]:
print "Random Forest Grid Search***"

feature_cols =  Procedure4feat + Procedure3feat +ClassFeat + NonScaledContfeat + sigCodesList

X= dfNotNull[feature_cols]
#n_ests = range(1, 15, 2)
#n_ests = range(1, 90, 2)
max_feats = range(1,len(feature_cols)+1)
#max_feats = range(2,13)
param_grid = dict(n_estimators=n_ests,max_features =max_feats)


rfreg = RandomForestRegressor()

grid = GridSearchCV(rfreg, param_grid, cv=5,scoring='mean_squared_error').fit(X,y)
print np.sqrt(-grid.best_score_)


Random Forest Grid Search***
10.634841712


In [134]:
print "BEST FEATURES FOR RANDOM FOREST GRID SEARCH AND PREDICTIONS"
print grid.best_estimator_
print np.sqrt(-grid.best_score_)
print pandas.DataFrame({'feature':feature_cols, 'importance':grid.best_estimator_.feature_importances_}).sort('importance')
RFPredictions =grid.best_estimator_.predict(X)

BEST FEATURES FOR RANDOM FOREST GRID SEARCH AND PREDICTIONS
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=6, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=9, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
10.634841712
                                  feature  importance
4                           Nerve Block 2    0.000161
12                            Nerve Block    0.000829
8                   Facet Joint Injection    0.001285
10  Cervical/Thoracic Medial Branch Block    0.001296
15                 Joint Injection, other    0.002101
7        Cervical/Thoracic Radiofrequency    0.002288
2                     Medial Branch Block    0.002528
9                   Cervical/Thoracic ESI    0.002616
3                               Injection    0.002629
27                                    Z87    0.003371
28                 

C:\Users\James\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [116]:
rfreg = RandomForestRegressor(n_estimators=41, max_features=7, oob_score=True)
rfreg.fit(X, y)
pandas.DataFrame({'feature':feature_cols, 'importance':rfreg.feature_importances_}).sort('importance')
X_important = rfreg.transform(X, threshold=.01)
rfreg.oob_score_

C:\Users\James\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
C:\Users\James\Anaconda2\lib\site-packages\sklearn\utils\__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


0.61799083143519873

In [38]:
"Grid search with cut down features"

print X_important


n_ests = range(1, 29, 2)
n_ests = range(1, 90, 2)
max_feats = range(1,len(X_important[0,])+1)
#max_feats = range(2,13)
param_grid = dict(n_estimators=n_ests,max_features =max_feats)


rfreg = RandomForestRegressor()

gridcut = GridSearchCV(rfreg, param_grid, cv=5,scoring='mean_squared_error').fit(X_important,y)
print np.sqrt(-gridcut.best_score_)




NameError: name 'X_important' is not defined

In [37]:

print" best_features from grid search, cut features"
print gridcut.best_estimator_
print np.sqrt(-gridcut.best_score_)


 best_features from grid search, cut features


NameError: name 'gridcut' is not defined

In [137]:
"Run Random Forest Again"
max_feats = 7

#max_feats = range(2,13)
param_grid = dict(n_estimators=[41],max_features =[max_feats])
grid = GridSearchCV(rfreg, param_grid, cv=5,scoring='mean_squared_error').fit(X,y)
print np.sqrt(-grid.best_score_)


NameError: name 'param_grid' is not defined

In [231]:
"My Transformer"


class EnsembleTranformer(TransformerMixin):
        def __init__(self, model, y):
            self.model = model
            self.y = y
        def transform(self,X,  **transform_params):
            if "KN" in str(self.model):
                X = X[knn_features]
                pass
            
            
            return np.array([self.model.predict(X)]).T
            
            
            
            #return X
           
# pipeline = feature union of EnsembleTranformer
        def fit(self, X, y=None, **fit_params):
            knn_features =  ['Lumbar Radiofrequency', 'Spinal Cord Stimulator', 'Lumbar ESI', '1r', 'Radiofrequency', 'ESI', 'Medial Branch Block', 'Spinal Cord Stimulator 2']
            if "KN" in str(self.model):
                
                X = X[knn_features]
                
                
            
            self.model.fit(X, y)
            return self

In [249]:
"LINEAR Ression Ensemble Sinan's"

"Dont Delete!"
"""from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
e = EnsembleTranformer(knn, response)
e.fit_transform(features)"""
"Dont Delete!"


knn = KNeighborsRegressor(n_neighbors=7)

y = dfNotNull['TotalTimeMin']
rf = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
    max_features=3, max_leaf_nodes=None, min_samples_leaf=1,
    min_samples_split=2, min_weight_fraction_leaf=0.0,
    n_estimators=51, n_jobs=1, oob_score=False, random_state=None,
    verbose=0, warm_start=False)


rf_features = feature_cols
X =  dfNotNull[feature_cols]

#e =EnsembleTranformer(knn, y)
#print e.fit_transform(dfNotNull[knn_features]).shape

linreg =  LinearRegression() 
linreg1 =  LinearRegression() 
base = (2,11.5,11.5)
for k in range(15,45):
    print k
    knn1 = KNeighborsRegressor(n_neighbors=k)
    pipe = Pipeline([
        ('union', FeatureUnion([
            ('knn',EnsembleTranformer(knn, y)),
            ('rf', EnsembleTranformer(rf, y)),
            ('linreg', EnsembleTranformer(linreg, y))
             ])) 
        ,('mdl', knn1 )
         ])

    mean2escores = np.sqrt(-cross_val_score(pipe, X, y, cv=5, scoring='mean_squared_error').mean())
    meanabscores = -cross_val_score(pipe, X ,y, cv=5, scoring='mean_absolute_error').mean()
    if meanabscores < base[2]:
        print k
        print mean2escores
        print meanabscores
        base = (k, mean2escores,meanabscores )
        
        
print "Scores here*****"
print base






15
15
10.8489012208
7.00817446687
16
16
11.04788415
6.88556851116
17
18
19
20
20
10.5921065088
6.83044063976
21
22
23
24
24
10.6024237459
6.81963110062
25
26
27
28
28
10.6964959513
6.77216616574
29
30
31
32
33
34
34
10.8426015238
6.76773703685
35
36
37
38
39
40
41
42
43
44
Scores here*****
(34, 10.842601523823049, 6.7677370368499483)


In [40]:
print "Grid Serch linreg ensemble, IN Progress!!!"

neighbs_k = range(1, 30)
param_grid = dict(neigh__n_neighbors=neighbs_k )
max_feats = range(2,13)
neigh = KNeighborsRegressor()

param_grid = dict(n_estimators=[41],max_features =[max_feats],neigh__n_neighbors=neighbs_k )

pipe = Pipeline([
    ('union', FeatureUnion([
        ('knn',EnsembleTranformer(knn, y)),
        ('rf', EnsembleTranformer(rf, y)),
         ])) 
    ,('mdl', knn1 )
     ])


grid2 = GridSearchCV(pipe, param_grid, cv=5, scoring='mean_squared_error').fit(X,y)
print np.sqrt(-grid2.best_score_)


Grid Serch linreg ensemble, IN Progress!!!


NameError: name 'EnsembleTranformer' is not defined

In [252]:
#pipe.fit(X,y)
#predictionDF = pandas.DataFrame()
#predictionDF['Predictions'] = pipe.predict(X)
#predictionDF['Actual'] = y

pipe = Pipeline([
        ('union', FeatureUnion([
            ('knn',EnsembleTranformer(knn, y)),
            ('rf', EnsembleTranformer(rf, y)),
            ('linreg', EnsembleTranformer(linreg, y))
             ])) 
        ,('mdl', knn1 )
         ])
base = (0,1,2,11.5,11.5)
X =dfNotNull[feature_cols]
for n in range(1,17):
    print "n = " + str(n)
    for k in range(8,24):
        for m in range(1,12):
            knn1 = KNeighborsRegressor(n_neighbors=k)
            rf = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
            max_features=m, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=n, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
            
            mean2escores = np.sqrt(-cross_val_score(pipe, X ,y, cv=5, scoring='mean_squared_error').mean())
            meanabscores = -cross_val_score(pipe, X, y, cv=5, scoring='mean_absolute_error').mean()
            if mean2escores < base[3]:
                print n
                print k
                print m
                print mean2escores
                print meanabscores
                base = (n,k,m,mean2escores,meanabscores)
print "SCORES HERE**********"                
print base              
                
            





n = 1
1
8
1
10.6090771788
6.84543562268
1
8
5
10.5710379705
6.9626197099
1
9
2
10.5322166767
7.02386917318
1
9
4
10.5284870586
6.89824985532
1
10
3
10.5174541033
7.02168032557
1
12
3
10.4672381788
6.93624246271
n = 2
n = 3
n = 4
n = 5
n = 6


KeyboardInterrupt: 

In [166]:
print type(X)
"Previous high scores and testing those scores"
(12, 11, 5, 10.014813816946187, 7.3007346551759387)


<class 'pandas.core.frame.DataFrame'>


In [39]:
"Predict on Out of Sample Data"
pipe.fit
dfOOS = pandas.read_csv('OutOS.csv')
X = dfOOS[knn_features]
y = dfOOS['TotalTimeMin']
OutOfSamplePredictions = linreg.predict()








NameError: name 'knn_features' is not defined

In [ ]:
"Definition ensemble"


[  16.67737377   26.88799409   52.18018974   19.45599251   67.41578911
   30.09675508   40.30737541   29.73746953   18.33956777   27.12545038
   18.93320849   16.9627015    94.36707765   54.19856817   44.34413228
   38.08289398   29.07297212   30.68729468   37.8140844    61.13714428
  146.56437337   43.42566043   31.99330426   29.14382882   56.01084363
   21.59309909   20.817447     52.80518376   36.98298739   25.34452823
   45.41268557   13.51958535   20.76200209   20.95158693   27.00982335
   39.71373469   26.65053781   28.47933141   85.54279801   52.5747712
   32.58694497   18.57702406   15.49288169   21.07310442   33.80868082
   19.78529598   40.93236942   29.1471948    16.22345168   39.62635985
   11.25983146   11.59305409   34.0671276    22.78795399   58.23532505
   46.6313203    46.03767959   22.85123722   22.13886836   36.53942812
   25.60297501   24.91748711   28.48690487   34.63977783   32.58694497
   44.9119984    41.73211313   46.71869514   30.92475097   30.44983839
   27.9

In [88]:
print """
Run Linear Regression With Avg length feature ****
"""
"""pipe = Pipeline([
        
        ('AverageProcedureTransformer', FeatureScalarTransformer),
        ('linreg', linreg )
         ])
X = dfNotNull[['Procedure3', 'TotalTimeMin']]
abse = []
sqrd = []


"""
abse = []
sqrd = []
for k in range(1,2):
    print k
    for i in range(1,50):
        y = dfNotNull['TotalTimeMin']
        X = dfNotNull[feature_cols + ['Procedure3']]
        features_train, features_test, response_train, response_test \
        = train_test_split(X, y)
        dfT = features_train
        dfT['TotalTimeMin']= response_train
        dfT['AverageProcTime']=dfT['Procedure3'].apply(lambda x: CleanDefs.createProcAvgs(x,dfT) )
        
        y = np.array([dfT['TotalTimeMin']]).T
        dfT2 = dfT
        dfT2 = dfT2.drop('TotalTimeMin',1)
        dfT2 = dfT2.drop('Procedure3',1)
        
        
        X = dfT2
        
        linreg =  LinearRegression()
        neigh = KNeighborsRegressor(n_neighbors=k)
        linreg.fit(X,y)

        
        dfR= features_test
        dfR['TotalTimeMin']= response_test
        dfR['AverageProcTime']=dfR['Procedure3'].apply(lambda x: CleanDefs.createProcAvgs(x,dfT) )
        

        dfR = dfR.fillna(response_train.mean())
        results = np.array([dfR['TotalTimeMin']]).T
        dfR = dfR.drop('TotalTimeMin',1)
        dfR = dfR.drop('Procedure3',1)
        X = dfR

        
        #print linreg.predict(X)
        predictions = []
        #for j in linreg.predict(X):
            #predictions+= [float(("%.5f" % j[0]))]
        pred = linreg.predict(X)[:,0].tolist()
        
        dfResults = pandas.DataFrame()
        #dfResults['Pred'] = np.array(predictions)
        dfResults['Pred'] = pred
        dfResults['Pred'] = dfResults['Pred'].astype('float').round(5)
        dfResults['Res']= results

        
        differences = (dfResults['Pred'] - dfResults['Res']).abs()
        abse += [differences.mean()]
        sqrd += [(differences**2).mean()]
    #print abse  
    print np.array(abse).mean()
    print np.sqrt(np.array(sqrd)).mean()
    


C:\Users\James\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\James\Anaconda2\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Run Linear Regression With Avg length feature ****

1
6.38432783387
9.95215311985


34.1856725146
F32 = 42.347826087, 23
F17 = 31.0612244898, 98
M96 = 43.0961538462, 104
Z85 = 40.0833333333, 36
Z87 = 37.2727272727, 22
Z95 = 37.5675675676, 37


In [118]:
print "Best Algorithms Best Scores!!!"

print """






Lingreg - **********

feature_cols =['Lumbar Radiofrequency', 'Radiofrequency', 'ESI', 'Nerve Block 2', 'Injection', 'Medial Branch Block', '3', '1r']


feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection', 'Medial Branch Block',  '3','1r']#
feature_cols += ['Gender'] +['F17'] + ['Age'] root mean squared error
10.1141795366
mean Abs Scores
6.40786359836



"Standard SDGC W/ GridSearch*************"
feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection', 'Medial Branch Block',  '3','1r']#
feature_cols += ['Gender'] +['F17'] + ['Age'] root mean squared error
10.1141795366
mean Abs Scores
6.40786359836


"get null scores- aerage values"
6.56750199776
10.2422627522


LINREG - Select KBest  *****************

['Lumbar Radiofrequency', 'Spinal Cord Stimulator', 'Lumbar ESI', 'Radiofrequency', 'ESI', 'Spinal Cord Stimulator 2']

Root mean squared error:
10.3549488744
Mean Abs Scores:
6.58723506741

KNN

['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection','Medial Branch Block' ]#
feature_cols += ['Spinal Cord Stimulator']

Root mean squared error:

10.3692137347


"""


Best Algorithms Best Scores!!!







Lingreg - **********

feature_cols =['Lumbar Radiofrequency', 'Radiofrequency', 'ESI', 'Nerve Block 2', 'Injection', 'Medial Branch Block', '3', '1r']


feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection', 'Medial Branch Block',  '3','1r']#
feature_cols += ['Gender'] +['F17'] + ['Age'] root mean squared error
10.1141795366
mean Abs Scores
6.40786359836



"Standard SDGC W/ GridSearch*************"
feature_cols =['Lumbar Radiofrequency',  'Radiofrequency', 'ESI','Nerve Block 2','Injection', 'Medial Branch Block',  '3','1r']#
feature_cols += ['Gender'] +['F17'] + ['Age'] root mean squared error
10.1141795366
mean Abs Scores
6.40786359836


"get null scores- aerage values"
6.56750199776
10.2422627522


LINREG - Select KBest  *****************

['Lumbar Radiofrequency', 'Spinal Cord Stimulator', 'Lumbar ESI', 'Radiofrequency', 'ESI', 'Spinal Cord Stimulator 2']

Root mean squared error:
10.3549488744
Mean Abs Sc